# implement c-means

In [38]:
import sys
import pandas as pd
import numpy as np
from  scipy.spatial.distance import pdist
import matplotlib 
from matplotlib import pyplot as plt
from scipy.spatial.distance import pdist
%matplotlib inline
sys.path.append("./../model/")
# from kmeans import kmeans

In [144]:
class cMeans(object):
    def __init__(self,data,centers,weight):
        self.centers = centers #列が各データで、行が各センターを表す
        self.pre_centers = centers # 評価関数で比較するために前のcentersをとっておく
        self.new_centers = np.zeros(self.centers.shape) #更新するためのcenters
        self.numC = self.centers.shape[1] #centerの列の数を表す
        self.data = data # 要素* データ数(列ベクトル)
        self.numD = self.data.shape[1] #dataの列にデータの個数が来る
        self.U = np.random.dirichlet(np.ones(self.numC),size=self.numD).T # numC行, numD列 縦にsumすると1.0である
        self.m = weight

    # calculate means
    def means(self):
        for i in range(self.numC):
            upper_sum = []
            under_sum = []
            for k in range(self.numD):
                upper_sum.append((self.U[i][k] **self.m)* self.data[:,k])
                under_sum.append(self.U[i][k] **self.m)
            result = (sum(upper_sum))/ (sum(under_sum))
            self.new_centers[:,i] = result #new_centersのi列にi番目のclusterセンターを代入
        return None
                
                
    # membership 
    def members(self):
        for k in range(self.numD):
            for i in range(self.numC):
                sum_tmp_0 = []
                for j in range(self.numC):
#                     diff_jk = self.data[:,k] - self.centers[:,j]
#                     diff_ik = self.data[:,k] - self.centers[:,i]
                    under = self.dist(self.data[:,k],self.centers[:,j]) #d_jk
                    upper = self.dist(self.data[:,k],self.centers[:,i]) #d_ik... j_for文内で定数
                    tmp_0 = (upper/under) ** (2/(self.m-1))
                    sum_tmp_0.append(tmp_0)
                result = (sum(sum_tmp_0)) ** (-1)
                self.U[i][k] = result
        return None

    
    # 引き数は計算前のmatrix
    def dist(self,y,v):
        # return norm
        return pdist([y,v],'minkowski', 2) 
    
    
    
    def getBestMedoidsAll(self,medoids):
        #全データからセントロイドまでの距離を測り、最短のものを返す。
        pair = []
        self.r = np.zeros([data.shape[0],self.init.shape[0]])
        for n_data in range(len(self.data)):
            iClu = self._getBestMedoids(one_data=data[n_data],medoids=medoids)
            #Rの更新を行う
            self.r[n_data,iClu-1] = 1
        return None

    
    def _getBestMedoids(self,one_data,medoids):
        #各点からセントロイドまでの距離を測って、最小のものを取り出す。
        tmp = []
        for j in range(len(medoids)):
            dis = self._distance(one_data,medoids[j])
            tmp.append(dis)
        result = min(tmp)
        index = tmp.index(result)
        return index


In [137]:
def _getInitValues(num,mini=-100,maxi=100):
    centroids = []
    if mini > maxi:
        raise ValueError('maxi should be greater than mini')
    else:
        for i in range(num):
            meanBtw = (maxi - mini)/ 2 + mini
            distance = maxi - meanBtw
            result = (np.random.rand(2) - 0.5) * distance * 2 + meanBtw
            centroids.append(result)
        return centroids

In [149]:
data = np.loadtxt("../data/faithful.txt")
data = np.array(data).T
#initial centroids
centroids = _getInitValues(3,mini=10,maxi=30)
centroids = np.array(centroids).T
# centroids = np.array(centroids)
cmeans = cMeans(data=data,centers=centroids,weight=2)

In [150]:
cmeans.means()